In [4]:
import os
import shutil

In [5]:
files = os.listdir("../../data/results")
files

['centers_using_ap.csv',
 'centers_using_dc.csv',
 'centers_using_si.csv',
 'convolution.csv',
 'convolution.png',
 'h1',
 'h10_short',
 'h11_short',
 'h12_short',
 'h13_short',
 'h14_short',
 'h15_short',
 'h16_short',
 'h17_short',
 'h18_short',
 'h19_short',
 'h1_try',
 'h2',
 'h20_short',
 'h21_short',
 'h22_short',
 'h23_short',
 'h24',
 'h25_short',
 'h3',
 'h4',
 'h5_short',
 'h6',
 'h7',
 'h7_short',
 'h8_short',
 'h9_short',
 'info_about_columns.md',
 'info_on_raw_data.csv',
 'info_on_smooth_data.csv',
 'ref',
 'ref.txt',
 'using_ap.png',
 'using_dc.png',
 'using_si.png',
 'x_y_using_ap.png',
 'x_y_using_dc.png',
 'x_y_using_si.png']

In [6]:
folders = [f for f in files if "." not in f]
folders

['h1',
 'h10_short',
 'h11_short',
 'h12_short',
 'h13_short',
 'h14_short',
 'h15_short',
 'h16_short',
 'h17_short',
 'h18_short',
 'h19_short',
 'h1_try',
 'h2',
 'h20_short',
 'h21_short',
 'h22_short',
 'h23_short',
 'h24',
 'h25_short',
 'h3',
 'h4',
 'h5_short',
 'h6',
 'h7',
 'h7_short',
 'h8_short',
 'h9_short',
 'ref']

In [7]:
original_file = "Ref_Notebook.ipynb"
for f in folders:
    new_file_name = f + ".ipynb"
    shutil.copy(original_file, new_file_name)

In [1]:
from life_saving_tools.files.notebook import Notebook

In [2]:
n = Notebook()

In [3]:
n.create_content("Ref_Notebook.ipynb")

Converting the Notebook to HTML.
Looking for the Headers and their positions.
Getting correct order of the Headers.
Creating the Contents.
Content copied to clipboard.
